In [38]:
import networkx as nx

In [39]:
def abrir_txt(nombre_archivo):
    archivo=open(nombre_archivo)
    data=[]
    for linea in archivo:
        linea=linea.strip()
        columna=linea.split()
        data.append(columna)
    return data

Agrego a la red data del género de los delfines.

In [40]:
Red_delfines = nx.read_gml('./data/dolphins.gml')
dolphins_gender = abrir_txt('./data/dolphinsGender.txt')
gender_dict = dict(dolphins_gender)
nx.set_node_attributes(Red_delfines, gender_dict, 'gender')

In [41]:
nodos = dict(Red_delfines.nodes(data = True))
masculinos = []
femeninos = []
sin_genero = []
for nodo in nodos:
    if gender_dict[nodo] == 'm':
        masculinos.append(nodo)
    if gender_dict[nodo] == 'f':
        femeninos.append(nodo)
    if gender_dict[nodo] == 'NA':
        sin_genero.append(nodo)

In [42]:
%store masculinos
%store femeninos
%store sin_genero
%store gender_dict

Stored 'masculinos' (list)
Stored 'femeninos' (list)
Stored 'sin_genero' (list)
Stored 'gender_dict' (dict)


Queremos ver la relación entre genéro y las comunidades halladas. 

In [43]:
print(f'Tenemos {round(len(masculinos) / Red_delfines.number_of_nodes() * 100, 2)}% de delfines masculinos, {round(len(femeninos) / Red_delfines.number_of_nodes() * 100, 2)}% de hembras y {round(len(sin_genero) / Red_delfines.number_of_nodes() * 100, 2)}% sin género.')

Tenemos 54.84% de delfines masculinos, 38.71% de hembras y 6.45% sin género.


Queremos ver si las comunidades halladas tienen la distribución de grado que se esperaría al hacer comunidades al azar (similar a las de la red total) o si tienen prevalencia de algún género. 

In [44]:
%store -r particiones

Nos guardamos un diccionario con los géneros y las particiones.

In [59]:
from collections import defaultdict
particiones_con_genero = {}

for metodo in particiones.keys():
    particiones_con_genero[metodo] = defaultdict(list)
    for d in (particiones[metodo], gender_dict): # you can list as many input dicts as you want here
        for key, value in d.items():
            particiones_con_genero[metodo][key].append(value)

In [76]:
dicc_genero_metodo_comunidad = {}

for metodo in particiones_con_genero:
    dicc_genero_metodo_comunidad[metodo] = {}
    cantidad_de_comunidades = max(particiones_con_genero[metodo].values(), key=lambda x: x[0])[0]
    for comunidad in range(0, cantidad_de_comunidades):
        dicc_genero_metodo_comunidad[metodo][comunidad] = {}
        dicc_genero_metodo_comunidad[metodo][comunidad]['Hombres'] = 0
        dicc_genero_metodo_comunidad[metodo][comunidad]['Mujeres'] = 0
        
        for delfin in particiones_con_genero[metodo]:
            if particiones_con_genero[metodo][delfin][0] == comunidad and particiones_con_genero[metodo][delfin][1] == 'm':
                dicc_genero_metodo_comunidad[metodo][comunidad]['Hombres'] += 1
            if particiones_con_genero[metodo][delfin][0] == comunidad and particiones_con_genero[metodo][delfin][1] == 'f':
                dicc_genero_metodo_comunidad[metodo][comunidad]['Mujeres'] += 1
                

In [116]:
df_genero = pd.concat({k: pd.DataFrame(v) for k, v in dicc_genero_metodo_comunidad.items()}).transpose()
df_genero

Edge_bet         Fast_greedy         Infomap         Louvline        
   Hombres Mujeres     Hombres Mujeres Hombres Mujeres  Hombres Mujeres
0      5.0     2.0         8.0    15.0     3.0     2.0      7.0     2.0
1      3.0    16.0         9.0     5.0     4.0    13.0     14.0     2.0
2      9.0     3.0        17.0     2.0     9.0     3.0      0.0     4.0
3     16.0     3.0         NaN     NaN     8.0     0.0      9.0     3.0
4      NaN     NaN         NaN     NaN    10.0     2.0      NaN     NaN

In [100]:
import scipy.stats as stats
generos_red = [len(masculinos), len(femeninos)]

fisher_dict = {}

for metodo in dicc_genero_metodo_comunidad:
    fisher_dict[metodo] = {}
    for comunidad in dicc_genero_metodo_comunidad[metodo]:
        generos_comu = [dicc_genero_metodo_comunidad[metodo][comunidad]['Hombres'], dicc_genero_metodo_comunidad[metodo][comunidad]['Mujeres']]
        _, pvalue = stats.fisher_exact([generos_comu, generos_red]) 
        fisher_dict[metodo][comunidad] = pvalue

In [117]:
df_fisher = pd.DataFrame.from_dict(fisher_dict)
df_fisher.index.name = 'Comunidad'
df_fisher.round(2)

,Edge_bet,Fast_greedy,Louvline,Infomap
Comunidad,,,,
0,0.69,0.08,0.46,1.00
1,0.00,0.77,0.04,0.01
2,0.35,0.01,0.04,0.35
3,0.05,NaN,0.35,0.04
4,NaN,NaN,NaN,0.19


# Aca me falta ver algo: es valido hacer un promedio de los p values para cada metodo??